# Course 1: Lidar
## Part 2: Point Cloud Segmentation 
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Sensor Fusion Nanodegree programme offered at Udacity.

## Objectives

* Use the [Point Cloud Library](https://pointclouds.org/) (PCL) to segment point clouds;
* Implement the [Random Sample Consensus](https://en.wikipedia.org/wiki/Random_sample_consensus) (RANSAC) algorithm for planar model fitting;
* Perform ground plane separation to "remove" the road surface from our point cloud data.

## 1. Introduction

### 1.1. Point Cloud Segmentation

Our goal in this lesson is to _process_ the point cloud data we received from the LiDAR sensor and _segment_ it in such a way that we are able to "remove" any of the points from our scene which do not belong to the obstacles we wish to detect. In the driving domain, one common type of points in which we wish to remove are _ground plane_ points. These are the points which "belong" to the road surface(s) that the vehicle(s) travel along. 

There are many potential use cases for LiDAR sensors in the automotive space, with point cloud segmentation being one such use case discussed in detail here. There are many other uses for LiDAR sensors and point clouds in the automotive and robotics space: multi-modal sensor calibration / fusion techniques, semi-supervised deep learning models for depth estimation, and for bench-marking tasks used to e.g., evaluate the performance of a "stand-alone" sensor on a given task (such as RGB camera-based vision systems on the monocular depth estimation task).

In the following section, we will narrow our scope to focus only on ground plane estimation as it relates to point cloud segmentation in the section below...

#### Ground Plane Estimation: Problem Setup

In this subsection we will review the topic of ground plane estimation, which is a preliminary exercise for point cloud segmentation and is essential to performing obstacle detection on LiDAR point cloud data. Ground plane estimation involves "detecting" and segmenting points in a point cloud which belong to the ground surface(s) in a scene. Using a technique known as planar fitting, we can extract the points from a levelled surface and remove them from the point cloud. Once these points are effectively discarded, we can begin to segment the object(s) or obstacle(s) for upstream tasks such as detection, tracking, classification, etc.

One primary assumption we will make about the surface we wish to segment is that it is level. In other words, we assume the ground plane / road surface our ego-vehicle is travelling on is _non-inclined_, i.e., that it has a $0^{\circ}$ elevation coinciding with the normal vector (usually defined as the vector perpendicular to the road surface; the upward-facing $z$-axis). To further simplify this problem, we also assume that the surface the ego-vehicle is travelling on is a uniform [planar surface](https://en.wikipedia.org/wiki/Planar_Riemann_surface). In other words, we assume there are no imperfections or deviations along the surface, which would cause the surface to deviate from a smooth, continuous composition. This implies that a constant elevation and constant slope exist along the surface's length and width. In the real-world, road surfaces have elevation, slope, and [textures](https://en.wikipedia.org/wiki/Road_texture) — design elements which break the simplifications we are forming for this toy problem. While real-world road surfaces complicate our rudimentary approach to road plane segmentation, advancements in R&D have been able to handle the non-uniform geometries that exist with non-planar road surfaces by employing various techniques such as deep learning feature extraction [1].   

In our toy example, we proceed with a planar surface assumption that will allow us to form a set of geometric constraints onto our environment which will help us better project a "surface" used to blanket the potential LiDAR points we wish to include in the segmented point cloud. Such a simplification will allow us to reduce the plane fitting estimation into an exercise of "best-fit". Using the [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus) algorithm, we can "work out" which points belong to the ground plane / simplified road surface by randomly-sampling a small set of points, fitting a "plane equation" to those sampled points, then performing an iterative two-step inlier "verification" and "consensus" stage to refine the planar surface estimation. It is with this iterative estimation algorithm we can determine a ground plane equation which "fits" or includes the most inlier (true positive) points, leaving us with a "segmentation" of sorts that includes most, if not all, the points belonging to our modelled road surface.

#### Further Considerations for Autonomous Vehicles

We discussed above the necessary trade-offs of our ground plane segmentation problem as it relates to real-world road surfaces. Now, we briefly address the additional constraints placed upon the solution space that come with applications designated for autonomous vehicles. One of those constraints is _robustness_; the ability of an algorithm to appropriately segment a multitude of road surfaces in a variety of conditions is crucial. One grid-based technique, known as multi-level segmentation, handles the diversity in road surfaces by breaking apart the point cloud into smaller chunks to analyse the subsections separately (a "divide-and-conquer" strategy). Another way to account for dynamic road surface characteristics is to use statistical methods to define local features or adjacent points (e.g., Iterative Closest Point). These approaches work by defining a set of parameters (e.g., distance thresholds, point densities, terrain characteristics) to better handle (or adapt) to these challenges. 

With any approach, a very real constraint must be considered. That is, the need for real-time processing. Efficient algorithms are critical, as autonomous vehicles must quickly and effectively perform segmentation on incoming LiDAR data. Therefore, scalable solutions are usually at the forefront of a researcher's mind when it comes to designing for autonomous vehicles.

#### Techniques for Ground Plane Estimation

As organised by T. Gomes [2] in the discussion presented (and lightly summarised by GPT) below, there are five main categories of ground plane estimation techniques that have been explored for the autonomous vehicle use case.

1. **Grid-based Segmentation**: this technique involves breaking up the road surface into 2.5D grid-space known as "tessellations". This grid-based representation of 3D space can reduce computational complexity and alleviate memory demands;
   * Elevation maps — each discretised cell in a "map" is assigned a value based on all points which surround it (e.g., mean average height); the elevation map technique relies on a surface gradient calculation, a clustering of adjacent points, and a "correction" step with respect to all points is calculated. Rudimentary elevation map-based approaches may fall short of real-time requirements and have shown limited ability to simultaneously model multiple surfaces somewhat frequently seen in the driving domain (e.g., road surface and the "terrain above a [freeway] underpass");
   * Occupancy grid maps — regarded as "extremely robust and easy to implement" [3], occupancy grid-based techniques exploit a trade-off between resolution and computational complexity (exploitation versus exploration). The occupancy grid approach helps handle the ambiguities that exists in noisy sensor data, accommodating for the various reflective properties (absorption and dispersion) of the surface being modelled;
2. **Ground Modelling with Object Maps**: this technique enforces geometric constraints and simplifications on the environment which allow the ground surface to be modelled by a set of geometric shapes or objects, such a single plane or a grid of planar surfaces. Our approach of planar fitting falls under this category;
   * Planar fitting — assumes that the ground surface can be modelled as a large, flat, uniform plane. Using estimation algorithms such as RANSAC allows for the efficient estimation of the plane's unknown parameters. However, it should be obvious that most road surfaces cannot be simply modelled as flat surfaces. Therefore, despite the computational efficiency, planar fitting with the RANSAC algorithm is not a robust enough solution to appropriately handle the complexities of the real world;
   * Line extraction — works by dividing the scene data into small patches. Within each patch, lines are fitted to capture the dominate linear trends. These lines can help identify edges or boundaries within the ground plane, such as curbs or changes in ground slope. Also apparent in this method its sensitivity to highly-uneven road surfaces, such as with off-road terrain found in the [DARPA Grand Challenge](https://en.wikipedia.org/wiki/DARPA_Grand_Challenge).
3. **Adjacent Points and Local Features**: these techniques explore the spatial relationships between neighbour points and local geometric features;
   * Channel-based — analyses the relationship between points contained in the vertical "channels" or bands of a LiDAR scan. Effectively handles uneven terrain but struggles in complex environments due to its dependence on parameterisation and its ignorance to the inherent horizontal channel-based relationships;
   * Region-growing — exploits a set of "similarity criteria" to fit a "region" around a seed point. While this method is easy to implement, results are highly dependent on choosing a good starting seed. Complex urban areas are one domain which this technique fails to handle effectively;
   * Clustering — groups point cloud data into segments or "clusters" based on height, or with features such as shape, or geometries such as surface normals. This voxelisation technique may be robust, it struggles in areas with low point-density and is computationally expensive to run over datasets with multi-dimensional features;
   * Range images — exploits "neighbourhood" relations (trigonometric relationships between adjacent points) found in 2D range images which have been converted from LiDAR point clouds. Real-time segmentation is possible using an angle-based approach, but accuracy can be dependent on the range image conversion process and through complexities related to the sensor model;
4. Higher-Order Inference: attempts to handle the sparsity in data which exists in LiDAR point clouds at farther distances;
   * Markov Random Field (MRF) — uses a probabilistic graph model to consider relationships between points in an undirected graph structure. Can be effective in handling sparse point clouds, but is computationally expensive and not robust enough to handle very rough terrain. A subset of MRF approaches aim to reduce computational complexity, but these algorithm- or data structure-dependent approaches suffer the same fate with respect to real-time demands;
   * Conditional Random Field (CRF) — captures long-range dependencies between points by labelling sequences of nodes given a chain of observations. Using temporal constraints, the spatio-temporal relationships are analysed to form a ground plane. Multiple iterations of CRF are required for uneven terrains, which comes at the cost of any real-time performance outlook.
5. Deep learning-based methods: a set of techniques using neural networks to perform segmentation, usually at the expense of computational efficiencies;
   * PointNet / PointNet++ — using methods formally suited for computer vision-related tasks, PointNet models extract local features with a hierarchical neural network designed to recognise fine-grained patterns and generalise to complex scenes. PointNet++ better learns from point clouds with varying point densities, but overall these methods require greater computational complexity in training and at inference time;
   * VoxelNet / PointPillar — discretises point clouds into voxels or vertical columns (pillars), while relying on a modified PointNet for the feature encoder module. Achieves good results, but requires additional computational complexity;
   * SqueezeSeg / RangeNet++ — utilises 2D representations (birds-eye view images), relies on a CRF backbone implemented as a recurrent layer. Achieves faster performance at the cost of accuracy;
   * GndNet — achieves real-time performance (55Hz) by converting the point cloud into a 2D grid, uses a modified PointNet and 2D CNN as the backbone.
   
   
In addition to the above, there are many other deep learning-based techniques which seek to optimise run-time efficiency using hardware accelerators (FPGAs; Lyu et al.) or hardware-dependent optimisation methods. However, the challenges regarding high computational demands and need for large amounts of labelled training data still remain. 

## 2. Programming Task

#### Ground plane estimation with RANSAC

To start, we will perform ground plane segmentation of our LiDAR point cloud data using the estimation technique proposed by Hu et al. [4]. This is a RANSAC-based solution which "fits" a geometrically-modelled ground plane to the largest set of "inliers" — LiDAR points estimated to be belonging to the plane based on a minimum distance threshold.

The RANSAC algorithm works to iteratively determine the "minimum" distance needed to safely assign the inlier points to the ground plane. Starting by assigning the lowest elevation points contained in each frame, the RANSAC algorithm then calculates the error (distance) of the points to the estimated plane to minimise over. Then, the iteration and consensus stage is performed, where the modelled plane containing the most inliers will be kept. On truly flat surfaces, such as the one we will model, RANSAC-based planar fitting methods have been shown to achieve an effectiveness of "around 81%" on the KITTI dataset [5].

#### Using the Point Cloud Library (PCL)

In our first part of the programming task, we use the built-in functions contained in the Point Cloud Library (PCL) to perform ground plane segmentation. In our [`SFND_Lidar_Obstacle_Detection`]() project codebase, we define a `pointProcessor` object which will contain the helper functions that call the built-in PCL functions repsonsible for the core task of segmentation. 

For further clarification, see the Point Cloud Library tutorial page on [planar segmentation with PCL](https://pointclouds.org/documentation/tutorials/planar_segmentation.html#planar-segmentation). Let's get started...

##### E1.2.1: `processPointClouds`

In the [`/src/environment.cpp`]() file, we create an object instance of the `processPointClouds` class. The choice to instantiate this object on the stack or the heap is up to you. In the `initCamera()` function we create the `pointProcessor` objects onto the heap by initialising the object pointers, then setting them to point to the memory addresses created with the `new` keyword. However, if you would like to instantiate the object onto the stack, then observe in `simpleHighway()` the declaration of the new `ProcessPointCloud` class objects.   

Defined in [`/src/processPointClouds.h`](), this object contains all the class functions needed to load / save, filter, extract and segment the point cloud objects using Point Cloud Library.  

```cpp
// In `/src/environment.cpp`:

void initCamera(
    CameraAngle setAngle,
    pcl::visualization::PCLViewer::Ptr& viewer
) {

  // ..
  /** E1.2.1: Create a point processor. **/
  // Here, we instantiate the `ProcessPointClouds` instances on the stack.
  // To instead use the heap, see `simpleHighway()`.
  // Instantiating two instances; one for 3D points,
  // the other for 3D + intensity-valued points.
  ProcessPointClouds<pcl::PointXYZ>* pointProcessor = new ProcessPointClouds<pcl::PointXYZ>();
  ProcessPointClouds<pcl::PointXYZI>* pointProcessor = new ProcessPointClouds<pcl::PointXYZI>();
}


void simpleHighway(
    pcl::visualization::PCLVisualizer::Ptr& viewer
) {
    /** E1.2.1: Create a point processor. **/
    // Here, we instantiate the `ProcessPointClouds` instances on the heap.
    // To instead use the stack, see `initCamera()`.
    // TODO:: Create point processor
    ProcessPointClouds<pcl::PointXYZ> pointProcessorXYZ;
    ProcessPointClouds<pcl::PointXYZI> pointProcessorXYZI;
```

## 3. Closing Remarks

#### Alternatives


#### Extensions of task

## 4. Future Work

* ⬜️ ;
* ⬜️ .

## Credits

This assignment was prepared by Aaron Brown and Michael Maile of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://learn.udacity.com/nanodegrees/nd313/)).


References
* [1] Paigwar, A. Erkent, Ö., Sierra-Gonzalez, D. Laugier, C. GndNet: Fast Ground Plane Estimation and Point Cloud Segmentation for Autonomous Vehicles. IEEE/RSJ International Conference on Intelligent Robots and Systems (IROS). 2020, pp. 2150-2156.
* [2] Gomes, T. Matias, Campos, D. Cunha, A. Roriz, R. A Survey on Ground Segmentation Methods for Automotive LiDAR Sensors. Sensors. 2023, 23(2):601. [doi:10.3390/s23020601](https://doi.org/10.3390/s23020601).
* [3] Thrun, S. Exploring artificial intelligence in the new millenium, chapter robotic mapping: a survey. Morgan Kaufmann. 2002, pp.1-35. 2002. [doi:10.5555/779343.779345](https://dl.acm.org/doi/10.5555/779343.779345).
* [4] Hu X., Florez S.A.R., Gepperth A. A Multi-Modal System for Road Detection and Segmentation. IEEE Intelligent Vehicles Symposium. 2014, pp.1365-1370. [HAL:hal-01023615](https://hal.science/hal-01023615).
* [5] Gieger, A. Lenz, P. Urtasun, R. Are we ready for autonomous driving? The KITTI vision benchmark suite. IEEE Conference on Computer Vision and Pattern Recognition. 2012. [doi:10.1109/CVPR.2012.6248074](https://ieeexplore.ieee.org/abstract/document/6248074).


Helpful resources:
* 